In [ ]:
%load_ext autoreload
%autoreload 2
#%matplotlib qt
%matplotlib widget

In [ ]:
import os
import sys
from pathlib import Path

src_dir = str(Path(os.getcwd()).parent / "src")
sys.path.insert(1, src_dir)
os.environ["PYTHONPATH"] = src_dir

In [ ]:
import data_loader
import calculate_concentr_diffs
from concentr_diffs_pathlength import *
from plotting import *
from mbll_functions import *
from utils import *
from mcs_function import *
import config
import pickle
from ray import tune
import matplotlib.pyplot as plt
from inspect import signature
from tqdm.notebook import tqdm

# Setup

In [ ]:
# unlabeled tissue (idx = 0) treated as gray matter
tissues_long = ["gray matter", "gray matter", "tumor", "blood vessel"]
tissues_short = tissues_long[1:]

load current reference hyperparameters

In [ ]:
try:
    with open(config.reference_params_path, "rb") as f:
        hyperparams = pickle.load(f)
except Exception:
    print("Could not load dictionary! Creating new one.")
    hyperparams = {}

In [ ]:
len(hyperparams)

In [ ]:
time_budget_s = 2700
num_samples = 2000

Create MCS attenuation object

In [ ]:
A_mcs = SimulationAttenuation(config.mcs_func_path)

In [ ]:
mcs_sparsity = 25

Load HELICoiD data

In [ ]:
loader = data_loader.DataLoaderHELICOID(
    config.dataset_dir,
    wavelength_left_cut=520,
    wavelength_right_cut=900
)

mu_a_matrix = loader.absorption_coefs(
    use_diff_oxycco=False,
    use_water_and_fat=True
)

Load Jacques parameters

In [ ]:
with open(config.m_params_path, "rb") as f:
    jacques_m_params, _, _, _ = pickle.load(f)


Parameter space

In [ ]:
c_ref_gray_matter_fractions = data_loader.DataLoader.tissue_parameters["gray matter"][0]

param_space = [
    # f_blood
    tune.sample_from(lambda _: min(1, max(0, np.random.normal(c_ref_gray_matter_fractions[0], 0.1)))),
    # st02
    tune.sample_from(lambda _: min(1, max(0, np.random.normal(c_ref_gray_matter_fractions[1], 0.25)))),
    # c_oxCCO
    tune.sample_from(lambda _: max(0, np.random.normal(c_ref_gray_matter_fractions[2], 0.002))),
    # c_redCCO
    tune.sample_from(lambda _: max(0, np.random.normal(c_ref_gray_matter_fractions[3], 0.002))),
    # f_water
    tune.sample_from(lambda _: min(1, max(0, np.random.normal(c_ref_gray_matter_fractions[4], 0.2)))),
    # f_fat
    tune.sample_from(lambda _: min(1, max(0, np.random.normal(c_ref_gray_matter_fractions[5], 0.2)))),
    # a
    tune.sample_from(lambda _: min(100, max(1, np.random.normal(data_loader.DataLoader.tissue_parameters["gray matter"][1][0], 10)))),
    # b
    tune.sample_from(lambda _: min(10, max(0.1, np.random.normal(data_loader.DataLoader.tissue_parameters["gray matter"][1][1], 1.5)))),
]

# Hyperparameter search

In [ ]:
nonlinear_models = {
    "Jacques, specific":
    (
        A_jacques_blood_fraction,
        {"const_vals": [list(jacques_m_params[tissue]) for tissue in tissues_short]}
    ),
    "Jacques, general":
    (
        A_jacques_blood_fraction, 
        {"const_vals": [list(jacques_m_params["general"])] * len(tissues_short)}
    ),
    "Carp":
    (
        A_carp_blood_fraction, 
        {
            "const_vals": [data_loader.DataLoader.tissue_parameters[tissue][-2:] for tissue in tissues_short],
        }
    ),
    "NLLS + MCS":
    (A_mcs.A_blood_fraction, {
        "jacobian": A_mcs.jacobian_blood_fraction
    })
}

Gute Bilder:
- 008-01
- 008-02
- 012-01
- 012-02
- 015-01
- 021-02
- 025-02

In [ ]:
models_to_search = list(nonlinear_models.keys())
#models_to_search = ["Jacques, specific", "Jacques, general", "Carp"]
#models_to_search = ["NLLS + MCS"]
models_to_search = ["Jacques, specific", "Jacques, general"]
# gute Bilder
image_ids = ["020-01", "015-01", "012-01", "008-01"]

In [ ]:
for image_id in image_ids:
    delta_A_gt_img = loader.get_attenuation_change(image_id)
    labeled_foreground_mask = (loader.label_map != 0) & (loader.label_map != 4)
    for model_str in models_to_search:
        model_func = nonlinear_models[model_str][0]
        print(f"Hyperparameter search for {model_str} and image {image_id}.")
        delta_A_gt_img_tmp, wavelengths_tmp, mu_a_matrix_tmp = delta_A_gt_img, loader.wavelengths, mu_a_matrix
        if model_str == "NLLS + MCS":
            delta_A_gt_img_tmp = delta_A_gt_img[::mcs_sparsity, :]
            wavelengths_tmp = loader.wavelengths[::mcs_sparsity]
            mu_a_matrix_tmp = mu_a_matrix[::mcs_sparsity, :]
        res_grid = concentr_fit_nonlinear_reference_param_search_multiple_tissues(
            delta_A_gt_img_tmp[:, labeled_foreground_mask],
            loader.label_map[labeled_foreground_mask], # only contains 0, 1, 2, 3 (unlabeled, normal, tumor, blood)
            wavelengths_tmp,
            mu_a_matrix_tmp,
            model_func,
            param_space,
            variables_bool_arr=([True] * 8) + ([False] * (len(signature(model_func).parameters) - 5)),
            left_bounds = np.array([0] * 8),
            right_bounds = np.array([1, 1, 0.05, 0.05, 1, 1, 100, 10]),
            update_init=True,
            constraint=None,
            init_vals=concentrations_to_blood_fraction(
                np.concatenate(data_loader.DataLoader.tissue_parameters["gray matter"][:2])
            ),
            spectra_per_report=20,
            grace_spectra=20,
            time_budget_s=time_budget_s,
            num_samples=num_samples,
            max_concurrent_trials=7,
            ref_val_tissue_idx=1, # reference pixel is gray matter, therefore use const_vals of row 1 for non-sampled values (e.g. g, n, m-params)
            **nonlinear_models[model_str][1]
        )

        score, configuration = get_best_values_from_res_grid(res_grid)
        cur_score, cur_configuration = hyperparams.get((image_id, model_str), (np.inf, None))
        if cur_score > score:
            hyperparams[(image_id, model_str)] = (score, configuration)
            print(f"Updating parameters for {model_str}")

            with open(config.reference_params_path, "wb+") as f:
                pickle.dump(hyperparams, f)
        else:
            print("Keeping old parameters.")

